#First time users stay time

以下の3通りに分けて初回起動ユーザーの平均起動時間を計算する
+ daily
+ weekly
+ monthly

##Input Parameters
+ DATE 集計期間の終わりの日
+ DEBUG 手動実行時のみTrue
+ FREQUENCY 実行頻度
+ BIGQUERY_PROJECT_ID bigqueryのプロジェクト名
+ BIGQUERY_DATASET bigqueryのデータセット
+ PACKAGE_NAME bigqueryのパッケージ名
+ OUTPUT_BIGQUERY_PROJECT_ID 出力先のBQのプロジェクト名
+ ISLATEST 最新の日付を対象にする場合はTrue,任意の日付を指定する場合はFalse

# Output Range
+ daily
DATEの1日前を対象に集計
 
  ex.DATE="2021-02-02"の場合は"2021-02-01を対象に集計"
+ weekly
DATEの1日前から7日を対象に集計

  ex.DATE="2021-02-22"の場合は"2021-02-15"から"2021-02-21を対象に集計"
+ monthly
DATEの1日前から1ヶ月を対象に集計

  ex.DATE="2021-02-01"の場合は"2021-01-01"から"2021-01-31"を対象に集計"

## Output Data
+ date　集計の開始日
+ android_first_users_stay_time	Android初回起動ユーザーの平均滞在時間
+ ios_first_users_stay_time	iOS初回起動ユーザーの平均滞在時間
+ all_first_users_stay_time 全ユーザーの平均滞在時間


# Parameters

In [1]:
DATE = "2021-01-01" # @param {type: "date"}
DEBUG = True # @param {type: "boolean"} 手動実行時のみTrueにする。Cloud FunctionsからFalseを渡される。
FREQUENCY = "monthly" # @param {type: "string"}
BIGQUERY_PROJECT_ID = "fl-komtar-herbert-offer" # @param {type: "string"}
BIGQUERY_DATASET = "analytics_211559993.events_*" # @param {type: "string"}
PACKAGE_NAME = "jp.co.hardoff.renk.app.offer" # @param {type: "string"}
OUTPUT_BIGQUERY_PROJECT_ID = "fl-komtar-analytics-dashboard" # @param {type: "string"}
IS_LATEST = True # @param {type:"boolean"}

# Constants

In [2]:
SESSION_TIMEOUT_MINS = 30 #@param {type:"number"}
METRICS_NAME = "first-time-users-stay-time"

# Version

In [3]:
VERSION = "7"

# Authorize

In [4]:
if DEBUG:
    from google.colab import auth
    auth.authenticate_user()

# Imports

In [5]:
import pandas as pd
import numpy as np
from datetime import timedelta,datetime
from pytz import timezone

# Get Input Datasets

## データの取得期間

In [6]:
if IS_LATEST:
  date = (datetime.now(timezone("Asia/Tokyo"))-timedelta(days=1))
else:
  date = datetime.strptime(DATE,"%Y-%m-%d") - timedelta(days=1)

if FREQUENCY == "daily":
  start = date
  end = date
elif FREQUENCY =="weekly":
  start = date-timedelta(days=6)
  end = date 
elif FREQUENCY == "monthly":
  end = date
  start = datetime(end.year,end.month,1)
else:
  raise Exception("Invalid frequency value")
start = start-timedelta(days=1)
end = end+timedelta(days=1)
start_date = start.strftime("%Y%m%d")
end_date = end.strftime("%Y%m%d")
start_date, end_date

('20210831', '20210909')

## GBQからデータを取得する

In [7]:
duration_func = """
var current = arr[0];
var result = [];
for (var i = 0; i < arr.length - 1; i ++) {
  var diff = arr[i + 1] - arr[i];
  if ((diff / 1000000) >= 1800) {
    result.push({
      event_timestamp: current,
      duration: (arr[i] - current) / 1000000,
    });
    current = arr[i + 1];       
  }
}
result.push({
  event_timestamp: current,
  duration: (arr[arr.length - 1] - current) / 1000000,
});
result = result.filter((r) => r.duration > 0)
return result;
"""
df_session_duration = pd.DataFrame(columns = ['date', 'Time', 'user_pseudo_id', 'user_first_touch_timestamp','First_touch_Time', 'OS', 'event_timestamp', 'duration'])

query = f"""
  CREATE TEMP FUNCTION
  DURATION(arr ARRAY<INT64>)
  RETURNS ARRAY<STRUCT<event_timestamp INT64,
  duration INT64>>
  LANGUAGE js AS '''
  {duration_func}
  '''; 
  WITH
      event_arr AS (
        SELECT
        FORMAT_TIMESTAMP("%Y%m%d", TIMESTAMP_MICROS(event_timestamp), 'Asia/Tokyo') AS date,
        user_pseudo_id,
        UPPER(platform)　as OS,
        user_first_touch_timestamp,
        DURATION(ARRAY_AGG(event_timestamp ORDER BY event_timestamp)) AS duration_arr
    FROM (
    SELECT * FROM `{BIGQUERY_DATASET}` 
    WHERE _table_suffix BETWEEN "{start_date}" AND "{end_date}"
    UNION ALL
    SELECT * FROM `{BIGQUERY_DATASET}` 
    WHERE _table_suffix BETWEEN "{start_date}" AND "{end_date}" AND app_info.id like "{PACKAGE_NAME}%"
    )
  GROUP BY
    date,
    user_pseudo_id,
    user_first_touch_timestamp,
    OS ),
  events AS (
    SELECT
      date,
      TIMESTAMP_MICROS(event_timestamp) AS Time,
      user_pseudo_id,
      user_first_touch_timestamp,
      TIMESTAMP_MICROS(user_first_touch_timestamp) AS First_touch_Time,
      OS,
      darr.event_timestamp,
      darr.duration
    FROM
      event_arr
    CROSS JOIN UNNEST(event_arr.duration_arr) AS darr
    WHERE date BETWEEN "{start_date}" AND "{end_date}"
  )
  SELECT *
  FROM
  events
  ORDER BY user_pseudo_id 
"""
df_gbq = pd.io.gbq.read_gbq(query, project_id = BIGQUERY_PROJECT_ID)
df_session_duration = pd.concat([df_session_duration, df_gbq])

df_session_duration = df_session_duration[(df_session_duration["date"] >= start_date) & (df_session_duration["date"] <= end_date)]
df_session_duration

,date,Time,user_pseudo_id,user_first_touch_timestamp,First_touch_Time,OS,event_timestamp,duration
0,20210901,2021-09-01 00:36:31.047000+00:00,000032b023cc46f658adb26edfc8a8c4,1599364212075000,2020-09-06 03:50:12.075000+00:00,ANDROID,1630456591047000,154
1,20210831,2021-08-31 02:27:30.335000+00:00,0001f154b6757877eb2107871365054a,1623117832849000,2021-06-08 02:03:52.849000+00:00,ANDROID,1630376850335000,2
2,20210908,2021-09-07 23:23:00.095000+00:00,0001f154b6757877eb2107871365054a,1623117832849000,2021-06-08 02:03:52.849000+00:00,ANDROID,1631056980095000,29
3,20210908,2021-09-08 01:11:22.755000+00:00,0001f154b6757877eb2107871365054a,1623117832849000,2021-06-08 02:03:52.849000+00:00,ANDROID,1631063482755000,788
4,20210908,2021-09-08 02:13:54.613000+00:00,0001f154b6757877eb2107871365054a,1623117832849000,2021-06-08 02:03:52.849000+00:00,ANDROID,1631067234613000,1
...,...,...,...,...,...,...,...,...
302496,20210902,2021-09-02 09:02:24.778000+00:00,fffef530cbd8c9d736eacfd1962937f0,1612065916072000,2021-01-31 04:05:16.072000+00:00,ANDROID,1630573344778000,21
302497,20210906,2021-09-06 08:48:43.597000+00:00,fffef530cbd8c9d736eacfd1962937f0,1612065916072000,2021-01-31 04:05:16.072000+00:00,ANDROID,1630918123597000,17
302498,20210908,2021-09-08 02:41:18.993000+00:00,fffef530cbd8c9d736eacfd1962937f0,1612065916072000,2021-01-31 04:05:16.072000+00:00,ANDROID,1631068878993000,10
302499,20210908,2021-09-08 10:14:04.777002+00:00,fffef530cbd8c9d736eacfd1962937f0,1612065916072000,2021-01-31 04:05:16.072000+00:00,ANDROID,1631096044777002,319


## 期間内の初回起動者を取得

In [8]:
unique_first_users = df_session_duration[df_session_duration["Time"]==df_session_duration["First_touch_Time"]]["user_pseudo_id"].unique()

## 期間内のユニークなユーザーの行取得

In [9]:
df_first_users_duration = df_session_duration[df_session_duration["user_pseudo_id"].isin(unique_first_users)]
df_first_users_duration

,date,Time,user_pseudo_id,user_first_touch_timestamp,First_touch_Time,OS,event_timestamp,duration
14,20210904,2021-09-04 11:08:09.465000+00:00,000579B0FB424A478FC3FC2B5035FC08,1630753689465000,2021-09-04 11:08:09.465000+00:00,IOS,1630753689465000,315
20,20210903,2021-09-03 03:02:12.308000+00:00,0007EF69E7A9463FAA45AC6B03FC1F79,1630638132308000,2021-09-03 03:02:12.308000+00:00,IOS,1630638132308000,95
21,20210903,2021-09-03 08:00:07.251000+00:00,0007EF69E7A9463FAA45AC6B03FC1F79,1630638132308000,2021-09-03 03:02:12.308000+00:00,IOS,1630656007251000,245
93,20210904,2021-09-04 06:16:29.315000+00:00,000b0c8ef4880b4d161c61f1ab78d6bb,1630736189315000,2021-09-04 06:16:29.315000+00:00,ANDROID,1630736189315000,749
186,20210907,2021-09-07 04:52:34.346000+00:00,00144A43A6A543F5B544DA78883C2A15,1630811736753000,2021-09-05 03:15:36.753000+00:00,IOS,1630990354346000,49
...,...,...,...,...,...,...,...,...
302044,20210904,2021-09-04 14:03:11.204000+00:00,ff34f4e94de7e23c24fd9035f4537089,1630764191204000,2021-09-04 14:03:11.204000+00:00,ANDROID,1630764191204000,52
302210,20210906,2021-09-06 06:43:58.571000+00:00,ff94bb995999187f8ec6862b135de476,1630910638571000,2021-09-06 06:43:58.571000+00:00,ANDROID,1630910638571000,74
302314,20210907,2021-09-07 04:53:50.191000+00:00,ffab2a18275414b46ae502eaec99380d,1630990430191000,2021-09-07 04:53:50.191000+00:00,ANDROID,1630990430191000,205
302350,20210905,2021-09-05 01:24:24.940000+00:00,ffc38073bc0bb18fbe7e16de71b3db89,1630756183616000,2021-09-04 11:49:43.616000+00:00,ANDROID,1630805064940000,1796


# All Device Duration

In [10]:
session_timeout = pd.Timedelta('%d min' % SESSION_TIMEOUT_MINS).total_seconds()
session_timeout

1800.0

In [11]:
session_duration_list = []
# ユーザ毎に滞在時間を求める。
for g_user_id, df_g_user in df_first_users_duration.groupby(["user_pseudo_id","OS"]):
  previous_row = None
  session_duration = pd.Timedelta('0 min')
# 各ユーザのイベントを1つ1つ確認する。
  for row in df_g_user.itertuples():
    # 一番初めのイベントの値を保存する。
    if previous_row is None:
      previous_row = row
      session_duration = pd.Timedelta(seconds = previous_row.duration)
      continue
    # 前後のイベント間隔が30分以上ひらいた時、滞在時間を求める。
    diff_session = datetime.fromtimestamp(row.event_timestamp/1000000)-datetime.fromtimestamp(previous_row.event_timestamp/1000000) #追加
    diff_session = diff_session.total_seconds() #追加
    if diff_session > session_timeout:
      session_duration_list.append({
          "date": previous_row.date, 
          "event_timestamp": previous_row.event_timestamp,
          "user_pseudo_id": g_user_id[0],
          "OS" : g_user_id[1], 
          "session_duration": session_duration,
      })
      session_duration = pd.Timedelta(seconds = row.duration)
    else:
        # 30分以下の場合は同じセッションに該当するとみなす。
      session_duration += pd.Timedelta(seconds = row.duration)
    previous_row = row
    # 一番最後のイベントを計測する。
  session_duration = pd.Timedelta(seconds = row.duration)
  session_duration_list.append({
      "date" : previous_row.date, 
      "event_timestamp": previous_row.event_timestamp,
      "user_pseudo_id" : g_user_id[0],
      "OS" : g_user_id[1], 
      "session_duration": session_duration,
  })
df_session_duration_list = pd.DataFrame(session_duration_list)
df_session_duration_list = df_session_duration_list.sort_values("date").reset_index(drop=True)
df_session_duration_list["session_duration"] = (df_session_duration_list.session_duration.astype(np.int64) / 1000000000).astype("int64")
df_session_duration_list.sort_values("user_pseudo_id")

,date,event_timestamp,user_pseudo_id,OS,session_duration
6527,20210904,1630753689465000,000579B0FB424A478FC3FC2B5035FC08,IOS,315
4188,20210903,1630638132308000,0007EF69E7A9463FAA45AC6B03FC1F79,IOS,95
4178,20210903,1630656007251000,0007EF69E7A9463FAA45AC6B03FC1F79,IOS,245
7406,20210904,1630736189315000,000b0c8ef4880b4d161c61f1ab78d6bb,ANDROID,749
8633,20210905,1630811736753000,00144A43A6A543F5B544DA78883C2A15,IOS,237
...,...,...,...,...,...
8632,20210905,1630826501666000,ff201caeaaecd8afcefbd2d9a97803a9,ANDROID,2164
5195,20210904,1630764191204000,ff34f4e94de7e23c24fd9035f4537089,ANDROID,52
11715,20210906,1630910638571000,ff94bb995999187f8ec6862b135de476,ANDROID,74
14235,20210907,1630990430191000,ffab2a18275414b46ae502eaec99380d,ANDROID,205


In [12]:
all_first_users_stay_time = df_session_duration_list["session_duration"].mean()
all_first_users_stay_time

518.319160752854

# Android

In [13]:
df_android_users = df_session_duration_list[df_session_duration_list["OS"]=="ANDROID"]
df_android_users

,date,event_timestamp,user_pseudo_id,OS,session_duration
3,20210831,1630412544030000,d41c6f85b51b4a19a710de20ffbd42ec,ANDROID,3211
4,20210831,1630398286325002,d41c6f85b51b4a19a710de20ffbd42ec,ANDROID,1010
5,20210831,1630394845310000,d41c6f85b51b4a19a710de20ffbd42ec,ANDROID,39
7,20210831,1630403261222000,129d9b4344152d8c1a8351865d33f588,ANDROID,141
13,20210831,1630398114982000,8a46703e7eaf14691a2d96352a132878,ANDROID,373
...,...,...,...,...,...
16177,20210908,1631090581410000,77a94814f072994a607c6a925bfdbb6d,ANDROID,64
16178,20210908,1631078347321000,77a94814f072994a607c6a925bfdbb6d,ANDROID,4
16182,20210908,1631075877300000,77a94814f072994a607c6a925bfdbb6d,ANDROID,138
16191,20210908,1631087242356000,31c21a5ad922ff39f8416f537afc2d0e,ANDROID,941


In [14]:
android_avg_session_duration = df_android_users["session_duration"].mean()
android_avg_session_duration

511.224567844577

# IOS

In [15]:
df_ios_users = df_session_duration_list[df_session_duration_list["OS"]=="IOS"]
df_ios_users

,date,event_timestamp,user_pseudo_id,OS,session_duration
0,20210831,1630402478009000,D813C2AFC61D4100A404A16DC0F72E45,IOS,282
1,20210831,1630405890090000,8EDE2BCAFD6743318FBDF747C052C5BB,IOS,1
2,20210831,1630406357365000,2E20F91DD742404D9D9F8B3B6A3D4D6C,IOS,38
6,20210831,1630417363218000,2E20F91DD742404D9D9F8B3B6A3D4D6C,IOS,25
8,20210831,1630395546159000,8F44A4CA2DD140DF87B7B7454442C3CE,IOS,260
...,...,...,...,...,...
16200,20210908,1631064939004000,611151331E784A55B2247BD5EF850E94,IOS,1520
16201,20210908,1631103356828000,131467D48CA744BF84DBD81BC4C58020,IOS,1966
16202,20210908,1631093428044000,8AFAE35D3BCA452DB10066B6E1BA0881,IOS,1199
16203,20210908,1631051258667000,52847F0BA9A5476A8201330A6F82D671,IOS,250


In [16]:
ios_avg_session_duration = df_ios_users["session_duration"].mean()
ios_avg_session_duration

523.4045967491459

# Output

In [17]:
df_output = pd.DataFrame(columns=["android_first_users_stay_time","ios_first_users_stay_time","all_first_users_stay_time"],index=[0])
df_output.insert(0, "date", start.strftime(format="%Y-%m-%d"))
df_output["date"] = pd.to_datetime(df_output["date"], format="%Y-%m-%d").dt.date
df_output["android_first_users_stay_time"] = android_avg_session_duration
df_output["ios_first_users_stay_time"] = ios_avg_session_duration
df_output["all_first_users_stay_time"] = all_first_users_stay_time
df_output = df_output.round(3)
df_output.to_gbq(f"""{PACKAGE_NAME.replace(".","_")}_{METRICS_NAME.replace("-","_")}.{FREQUENCY}_events_{start.strftime(format="%Y-%m-%d").replace("-","")}""",
                 if_exists="replace",
                 table_schema=[{'name': 'date','type': 'DATE'},
                               {'name': 'android_first_users_stay_time','type': 'FLOAT64'},
                               {'name': 'ios_first_users_stay_time','type': 'FLOAT64'},
                               {'name': 'all_first_users_stay_time','type': 'FLOAT64'}
                               ],
                 project_id=OUTPUT_BIGQUERY_PROJECT_ID)
df_output

1it [00:05,  5.19s/it]


,date,android_first_users_stay_time,ios_first_users_stay_time,all_first_users_stay_time
0,2021-08-31,511.225,523.405,518.319
